In [1]:
Identifier_and_Name = {
    "Au-PeEL": "Ebook Library",
    "AU-PeEL": "Ebook Library",
    "CaONFJC": "MyiLibrary",
    "CaPaEBR": "Ebrary",
    "DE-He213": "Springer-Verlag",
    "MiAaPQ": "ProQuest",
    "NhCcYBP": "YBP",
    "OCoLC": "OCLC",
    "SFPDA_MiAaPQ": "ProQuest",
    "YBPUID": "YBP ID",
    "MIL": "MyiLibrary",
    "EBR": "Ebrary",
    "ebrary": "Ebrary (full name)",
    "EBZ": "EBSCO",
    "EBL": "Ebook Library (unofficial)",
    "EBC": "Ebook Central",
    "CaBNVSL": "SIAM",
    "CaBNvSL": "SIAM",
    "DcWaAPA": "APA",
    "DNLM": "US Med Library",
    "FR-PaOEC": "OECD",
    "IN-ChSCO": "Scope e-Knowedge Center",
    "MiFhGG": "Gale",
    "StDuBDS": "Bibliographic Data Services",
    "UtOrBLW": "Backstage Library Works",
    "VaAlASP": "Alexander Street Press",
    "EBS": "EBSCO",
    "FANhCcYBP": "YBP (FA)",
    "UK-OxUP": "Oxford UP",
    "FlBoTFG": "T&F",
    "UkCbUP": "Cambridge UP",
    "no_label": "Unlabeled",
    "BAA": "BAA",
}

> Start by loading Alpeh Sequential text file into OpenRefine with column widths 10,6,2

Put the unique name of the OpenRefine project for the BIB records in the variable `BIB_Project_Name` and run the cell

In [2]:
BIB_Project_Name = "tf_bib_2021 05 19 txt"

# Organize BIB Record

## Organize MARC Fields and Subfields
Copy Organize_Alpeh_Sequential_for_BIB.json and paste into OpenRefine

## Pivot Subfields into Columns
Use a filter on "No Field Number" to select the following subfields, invert, then remove all matching rows:

- 020$a
- 020$q
- 020$z
- 024$2
- 024$a
- 035$a
- 245$a
- 245$b
- 245$c
- 245$n
- 250$a
- 264$b
- 264$c
- 710$a
- 710$e
- 776$i
- 776$z
- 856$3
- 856$u
- 856$z
- 897$a
- 897$e

Open a text filter for "Fields," click on the number of choices to get the facet choices as TSV, paste into variable "Columns" below, and run cell

In [3]:
Columns = """
020$a	15
020$q	2
024$2	370
024$a	885
035$a	3308
245$a	3831
245$b	976
245$c	1169
245$n	21
250$a	227
264$b	2783
264$c	1830
776$i	944
776$z	1543
856$3	885
856$u	1956
856$z	1288
Field #1 020$a	1105
Field #1 020$q	601
Field #1 020$z	5
Field #1 035$a	523
Field #1 264$b	895
Field #1 264$c	731
Field #1 776$i	1297
Field #1 776$z	123
Field #1 856$u	1875
Field #1 856$z	1875
Field #10 020$a	24
Field #10 020$q	56
Field #10 020$z	82
Field #10 856$u	43
Field #10 856$z	43
Field #11 020$a	18
Field #11 020$q	22
Field #11 020$z	21
Field #11 856$u	31
Field #11 856$z	31
Field #12 020$a	12
Field #12 020$q	13
Field #12 020$z	15
Field #12 856$u	22
Field #12 856$z	22
Field #13 020$a	2
Field #13 020$q	4
Field #13 020$z	4
Field #13 856$u	13
Field #13 856$z	13
Field #14 020$a	2
Field #14 020$q	2
Field #14 020$z	2
Field #14 856$u	10
Field #14 856$z	10
Field #15 020$a	1
Field #15 020$q	1
Field #15 020$z	1
Field #15 856$u	9
Field #15 856$z	9
Field #16 020$a	1
Field #16 020$q	1
Field #16 856$u	7
Field #16 856$z	7
Field #17 020$q	1
Field #17 020$z	1
Field #17 856$u	6
Field #17 856$z	6
Field #18 020$z	1
Field #18 856$u	2
Field #18 856$z	2
Field #19 020$z	1
Field #19 856$u	2
Field #19 856$z	2
Field #2 020$a	858
Field #2 020$q	587
Field #2 020$z	252
Field #2 035$a	523
Field #2 264$b	496
Field #2 264$c	383
Field #2 776$i	287
Field #2 776$z	331
Field #2 856$u	1875
Field #2 856$z	1875
Field #20 020$z	1
Field #20 856$u	2
Field #20 856$z	2
Field #21 856$u	2
Field #21 856$z	2
Field #22 856$u	2
Field #22 856$z	2
Field #23 856$u	2
Field #23 856$z	2
Field #24 856$u	2
Field #24 856$z	2
Field #25 856$u	1
Field #25 856$z	1
Field #26 856$u	1
Field #26 856$z	1
Field #27 856$u	1
Field #27 856$z	1
Field #3 020$a	454
Field #3 020$q	389
Field #3 020$z	459
Field #3 035$a	499
Field #3 264$b	108
Field #3 264$c	4
Field #3 776$i	49
Field #3 776$z	34
Field #3 856$u	1108
Field #3 856$z	1108
Field #4 020$a	404
Field #4 020$q	283
Field #4 020$z	407
Field #4 035$a	1
Field #4 264$b	24
Field #4 776$i	11
Field #4 776$z	22
Field #4 856$u	665
Field #4 856$z	665
Field #5 020$a	348
Field #5 020$q	258
Field #5 020$z	235
Field #5 264$b	8
Field #5 776$i	3
Field #5 856$u	399
Field #5 856$z	399
Field #6 020$a	336
Field #6 020$q	193
Field #6 020$z	173
Field #6 264$b	3
Field #6 776$i	2
Field #6 856$u	235
Field #6 856$z	235
Field #7 020$a	154
Field #7 020$q	188
Field #7 020$z	41
Field #7 856$u	138
Field #7 856$z	138
Field #8 020$a	151
Field #8 020$q	175
Field #8 020$z	37
Field #8 856$u	102
Field #8 856$z	102
Field #9 020$a	31
Field #9 020$q	116
Field #9 020$z	126
Field #9 856$u	63
Field #9 856$z	63
"""

**Indicator columns deleted at this point--unsure how to handle/how or if to preserve beyond this point**

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

In [4]:
Columns = Columns.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Column_Names = []
for Column in Columns:
    Column = Column.rstrip("1234567890")
    Column = Column.rstrip("\t")
    Column_Names.append(Column)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    for Column in Column_Names:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Column}",
                "expression": "grel:row.record.cells[\\"{Column}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{Column}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Column}",
                "description": "Blank down cells in column ``{Column}``"
            }},
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

In [5]:

def Column_Order_as_Letter(MARC_Tag):
    if MARC_Tag == "020$a":
        return "A"
    elif MARC_Tag == "020$z":
        return "B"
    elif MARC_Tag == "776$z":
        return "C"
    elif MARC_Tag == "020$q":
        return "D"
    elif MARC_Tag == "776$i":
        return "E"
    elif MARC_Tag == "024$a":
        return "F"
    elif MARC_Tag == "024$2":
        return "G"
    elif MARC_Tag == "035$a":
        return "H"
    elif MARC_Tag == "710$a":
        return "I"
    elif MARC_Tag == "710$e":
        return "J"
    elif MARC_Tag == "897$a":
        return "K"
    elif MARC_Tag == "897$e":
        return "L"
    elif MARC_Tag == "856$u":
        return "M"
    elif MARC_Tag == "856$z":
        return "N"
    elif MARC_Tag == "856$3":
        return "O"
    elif MARC_Tag == "264$c":
        return "P"
    elif MARC_Tag == "264$b":
        return "Q"
    elif MARC_Tag == "245$c":
        return "R"
    elif MARC_Tag == "250$a":
        return "S"
    elif MARC_Tag == "245$n":
        return "T"
    elif MARC_Tag == "245$b":
        return "U"
    elif MARC_Tag == "245$a":
        return "V"
    # No "else" statement because if there's a problem here, it will require manual intervention

Column_Order = {
    "A": {},
    "B": {},
    "C": {},
    "D": {},
    "E": {},
    "F": {},
    "G": {},
    "H": {},
    "I": {},
    "J": {},
    "K": {},
    "L": {},
    "M": {},
    "N": {},
    "O": {},
    "P": {},
    "Q": {},
    "R": {},
    "S": {},
    "T": {},
    "U": {},
    "V": {},
}

# Create nested dictionary for ordering columns
for Column in Column_Names:
    if " " in Column: # Has field number
        Field_Number = int(Column.split(" ")[1][1:])
        Ordering_Letter = Column_Order_as_Letter(Column.split(" ")[2])
        Column_Order[Ordering_Letter][Field_Number] = Column
    else:
        Ordering_Letter = Column_Order_as_Letter(Column)
        Column_Order[Ordering_Letter][0] = Column

# Create markers to ensure that all first column in pivots exist
if not Column_Order["Q"].get(0): # If there's no column that's just "264$b"
    Column_Order["Q"][0] = "264$b_marker"
if not Column_Order["P"].get(0): # If there's no column that's jsut "264$c"
    Column_Order["p"][0] = "264$c_marker"
if not Column_Order["O"].get(0): # If there's no column that's just "856$u"
    Column_Order["O"][0] = "856$u_marker"
if not Column_Order["H"].get(0): # If there's no column that's just "035$a"
    Column_Order["H"][0] = "035$a_marker"
if not Column_Order["A"].get(0): # If there's no column that's just "020$a"
    Column_Order["A"][0] = "020$a_marker"

# Sort by field order within tags
for key, value in Column_Order.items():
    value = dict(sorted(value.items()))
    Column_Order[key] = value

# Create list of column values and markers for columns needed for pivots in order
Column_Order_Values = []
for Outer_Value in Column_Order.values():
    for Inner_Value in Outer_Value.values():
        Column_Order_Values.append(Inner_Value)

# Create dictionary for adding columns needed for pivots and list of existing columns for adding to JSON 
Pivot_Addition_Column_Indexes = {Column_Value[0:5]: Column_Order_Values.index(Column_Value)+2 for Column_Value in Column_Order_Values if "_marker" in Column_Value}
Column_Order_Values = ['"' + Column_Value + '"' for Column_Value in Column_Order_Values if "_marker" not in Column_Value] #if "_marker" not in Column_Value
Reordering_String = ",\n".join(Column_Order_Values)

# Create final JSON for ordering columns
with open('JSON_Output.txt', 'w') as For_OpenRefine:
    Template_String = f"""
        [
            {{
                "op": "core/column-addition",
                "description": "Create column ``Count``",
                "engineConfig": {{
                    "facets": [],
                    "mode": "record-based"
                }},
                "newColumnName": "Count",
                "columnInsertIndex": 1,
                "baseColumnName": "SYS Number",
                "expression": "grel:row.record.cells[\\"SYS Number\\"].value[0]+\\"-\\"+toString(row.index-row.record.fromRowIndex+1)",
                "onError": "set-to-blank"
            }},
            {{
                "op": "core/column-reorder",
                "columnNames": [
                    "SYS Number",
                    "Count",
                    {Reordering_String}
                ],
                "description": "Reorder columns"
            }}
    """
    For_OpenRefine.write(Template_String)
    for Column_Name, Column_Position in Pivot_Addition_Column_Indexes.items():
        Template_String = f"""
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "record-based"
                }},
                "baseColumnName": "SYS Number",
                "expression": "grel:null",
                "onError": "set-to-blank",
                "newColumnName": "{Column_Name}",
                "columnInsertIndex": {Column_Position},
                "description": "Create column ``{Column_Name}`` for pivoting"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Organize_Rarely_Duplicated_Fields.json and paste into OpenRefine

Confirm no columns with rarely duplicated information remain; if any do, move the information to the appropriate column and delete the original column

Use text filters to standardize and clean up the contents of the "Edition" and "Volume" columns

## Organize Publisher and Pub Date Fields

**Multiple publisher fields handled by combining htem with `]|[` seperator; is there a better option?**

Copy Organize_Publisher_Fields.json and paste into OpenRefine

**PUB DATE ABLE TO HANDLE SAME DATE MULTIPLE TIMES--NEED TO FIGURE OUT STRATEGY FOR IF WE WANT TO PRESERVE BOTH DATES**

Copy Organize_Pub_Date_Fields.json and paste into OpenRefine

## Organize URLs

Copy Organize_URL_Fields.json and paste into OpenRefine
* If the row's value in "Fields" is null, the value in "Values" won't be retained through the pivot

Open a text filter for "Fields", perform any clustering if appropriate, click on the number of choices to get the facet choices as TSV, paste into variable "URL_Headers" below, and run cell


In [6]:
URL_Headers = """
856$u	244
Taylor & Francis	856
"""

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

Set "i" to the column number of column "020$a" and run cell

In [7]:
URL_Headers = URL_Headers.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Header_Names = []
for Header in URL_Headers:
    Header = Header.rstrip("1234567890")
    Header = Header.rstrip("\t")
    Header_Names.append(Header)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    i = 9
    for Header in Header_Names:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Header}",
                "expression": "grel:row.record.cells[\\"{Header}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{Header}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Header}",
                "description": "Blank down cells in column ``{Header}``"
            }},
            {{
                "op": "core/column-move",
                "columnName": "{Header}",
                "index": {i},
                "description": "Move column ``{Header}`` to position {i}"
            }}
        """
        For_OpenRefine.write(Template_String)
        i += 1
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

**897 and 710 fields not handled**

## Organize 035$a Identifier Fields

Copy Organize_035_Identifier_Fields.json and paste into OpenRefine

Open a text filter for "Fields", evaluate any remaining options with asterists at the end, and remove the asterisk if the label is to remain or the label if the value doesn't need to be retaiend

In the text filter for "Fields", click on the number of choices to get the facet choices as TSV, paste into variable "Identifier_List" below, and run cell

In [8]:
Identifier_List = """
BAA	1
FlBoTFG	564
NhCcYBP	499
no_label	12
OCoLC	34
YBPUID	512
"""

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

In [9]:
Identifier_List = Identifier_List.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Identifiers = []
for Identifier in Identifier_List:
    Identifier = Identifier.rstrip("1234567890")
    Identifier = Identifier.rstrip("\t")
    Identifiers.append(Identifier)

Old_and_New_Headers = dict()
for Identifier in Identifiers:
    if len(Identifier.split(" ")) == 1:
        if Identifier_and_Name.get(Identifier) is None:
            print(f"{Identifier} doesn't have a value. Add one to the dictionary Identifier_and_Name at the top of the notebook and rerun all cells.")
            continue
        Old_and_New_Headers[Identifier] = Identifier_and_Name.get(Identifier)
    else: # This handles when multiple IDs with the same org code exist in a record
        Repeated_Org_Code = Identifier.split(" ")[0]
        if Identifier_and_Name.get(Repeated_Org_Code) is None:
            # No statement output because it would be repetitive
            continue
        Column_Name_Base = Identifier_and_Name.get(Repeated_Org_Code)
        Old_and_New_Headers[Identifier] = Column_Name_Base + " " + Identifier.split(" ")[1]

Values_as_List = [Future_Column_Headers for Future_Column_Headers in Old_and_New_Headers.values()]
Values_as_Set = set(Values_as_List)
if len(Values_as_Set) < len(Values_as_List):
    Duplicates_Check = dict()
    for Set_Value in Values_as_Set:
        Duplicates_Check[Set_Value] = 0
        for List_Value in Values_as_List:
            if Set_Value == List_Value:
                Duplicates_Check[Set_Value] +=1

    Repeated_Headers = []
    for Key, Value in Duplicates_Check.items():
        if Value > 1:
            Repeated_Headers.append(Key)
    print("The headers below were matched to multiple org codes in Identifier_List:")
    print(Repeated_Headers)


BAA doesn't have a value. Add one to the dictionary Identifier_and_Name at the top of the notebook and rerun all cells.


Set `i` to the column number of column "Author"

In [11]:

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    i = 8
    for Key, Value in Old_and_New_Headers.items():
        Template_String = f"""
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                "facets": [],
                "mode": "row-based"
                }},
                "baseColumnName": "{Key}",
                "expression": "grel:row.record.cells[\\"{Key}\\"].value[0]",
                "onError": "set-to-blank",
                "newColumnName": "{Value}",
                "columnInsertIndex": {i},
                "description": "Create column ``{Value}`` by filling up and down ``{Key}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Value}",
                "description": "Blank down cells in column ``{Value}``"
            }},
            {{
                "op": "core/column-removal",
                "columnName": "{Key}",
                "description": "Remove column ``{Key}``"
            }}
        """
        For_OpenRefine.write(Template_String)
        i += 1
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

## Organize Other Identifier ISBN_Fields

Copy Organize_Other_Identifier_Fields.json and paste into OpenRefine

Open a text filter for "Fields", change any field names if appropriate, click on the number of choices to get the facet choices as TSV, paste into variable "Other_Identifiers_List" below, and run cell

In [12]:
Other_Identifiers_List = """
DOI	24
"""

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

Set `i` to the column number of the first URL column

In [13]:
Other_Identifiers_List = Other_Identifiers_List.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Other_Identifiers = []
for Other_Identifier in Other_Identifiers_List:
    Other_Identifier = Other_Identifier.rstrip("1234567890")
    Other_Identifier = Other_Identifier.rstrip("\t")
    Other_Identifiers.append(Other_Identifier)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    i = 13
    for Column_Name in Other_Identifiers:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Column_Name}",
                "expression": "grel:row.record.cells[\\"{Column_Name}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{Column_Name}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Column_Name}",
                "description": "Blank down cells in column ``{Column_Name}``"
            }},
            {{
                "op": "core/column-move",
                "columnName": "{Column_Name}",
                "index": {i},
                "description": "Move column ``{Column_Name}`` to position {i}"
            }}
        """
        For_OpenRefine.write(Template_String)
        i += 1
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

## Organise ISBNs

Copy Organize_ISBN_Fields_1.json and paste into OpenRefine

Perform clustering on column "ISBN Type"

Copy Organize_ISBN_Fields_2.json and paste into OpenRefine

Open a text filter for "Fields", click on the number of choices to get the facet choices as TSV, paste into variable "ISBN_Fields" below, and run cell

In [14]:
ISBN_Types = """
eISBN [vol. 4] 10	1
eISBN [vol. 4] 13	1
eISBN 10	58
eISBN 10 no 2	40
eISBN 10 no 3	18
eISBN 10 no 4	4
eISBN 13	60
eISBN 13 no 2	41
eISBN 13 no 3	18
eISBN 13 no 4	5
EPUB eISBN 10	6
EPUB eISBN 13	6
HB ISBN 10	10
HB ISBN 13	16
ISBN 10	37
ISBN 10 no 2	4
ISBN 13	51
ISBN 13 no 2	6
ISBN 13 no 3	2
Mobi eISBN 10	2
Mobi eISBN 13	2
MyiLibrary eISBN 10	1
MyiLibrary eISBN 13	1
PB ISBN 10	2
PB ISBN 13	2
PDF eISBN 10	6
PDF eISBN 13	6
"""

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

In [15]:
ISBN_Types = ISBN_Types.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
ISBN_Fields = []
for ISBN_Type in ISBN_Types:
    ISBN_Type = ISBN_Type.rstrip("1234567890")
    ISBN_Type = ISBN_Type.rstrip("\t")
    ISBN_Fields.append(ISBN_Type)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    for ISBN_Type in ISBN_Fields:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{ISBN_Type}",
                "expression": "grel:row.record.cells[\\"{ISBN_Type}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{ISBN_Type}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{ISBN_Type}",
                "description": "Blank down cells in column ``{ISBN_Type}``"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

# Organize HOL

> Start by loading Alpeh Sequential text file into OpenRefine with column widths 10,5,3

Put the unique name of the OpenRefine project for the HOL records in the variable `HOL_Project_Name` and run the cell

In [16]:
HOL_Project_Name = "wiley_hol_pt2_2021 05 17 txt"

Copy Organize_Aleph_Sequential_for_HOL.json and paste into OpenRefine

**Check retention of STA$a field presence**

Open a text filter for "Fields", click on the number of choices to get the facet choices as TSV, paste into variable `Number_of_TKRs` below, and run cell (if there aren't any TKRs, do this same procedure with `Number_of_TKRs` as an empty string)

In [17]:
Number_of_TKRs = """
TKR 1	1111
TKR 2	4
"""

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

In [18]:
Number_of_TKRs = Number_of_TKRs.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
TKR_Fields = []
for TKR in Number_of_TKRs:
    TKR = TKR.rstrip("1234567890")
    TKR = TKR.rstrip("\t")
    TKR_Fields.append(TKR)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    for TKR_Field in TKR_Fields:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{TKR_Field}",
                "expression": "grel:row.record.cells[\\"{TKR_Field}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{TKR_Field}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{TKR_Field}",
                "description": "Blank down cells in column ``{TKR_Field}``"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run the cell above, then copy JSON_Output.txt and paste into OpenRefine

Copy Organize_TKRs.json and paste into OpenRefine


## Add HOL data to BIB project
Switch back to BIB project and set `i` to position of first identifier column

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

In [19]:
i = 7

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    Template_String = f"""
        [
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "record-based"
                }},
                "baseColumnName": "SYS Number",
                "expression": "grel:cell.cross(\\"{HOL_Project_Name}\\",\\"BIB Number\\").cells[\\"SYS Number\\"].value.join(\\"]|[\\")",
                "onError": "set-to-blank",
                "newColumnName": "HOL Number",
                "columnInsertIndex": {i},
                "description": "Create column ``HOL Number`` with all the HOL numbers for the BIB seperated by `]|[`"
            }},
            {{
                "op": "core/multivalued-cell-split",
                "columnName": "HOL Number",
                "keyColumnName": "SYS Number",
                "mode": "separator",
                "separator": "]|[",
                "regex": false,
                "description": "Split values in column ``HOL Numbers`` into new rows at `]|[`"
            }},
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "row-based"
                }},
                "baseColumnName": "HOL Number",
                "expression": "grel:cell.cross(\\"{HOL_Project_Name}\\",\\"SYS Number\\").cells[\\"HOL Sublibrary\\"].value[0]",
                "onError": "set-to-blank",
                "newColumnName": "Sublibrary",
                "columnInsertIndex": {i+1},
                "description": "Create column ``Sublibrary`` with the sublibrary value from row with same BIB in HOL project"
            }},
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "row-based"
                }},
                "baseColumnName": "HOL Number",
                "expression": "grel:cell.cross(\\"{HOL_Project_Name}\\",\\"SYS Number\\").cells.Suppressed.value[0]",
                "onError": "set-to-blank",
                "newColumnName": "Suppressed",
                "columnInsertIndex": {i+2},
                "description": "Create column ``Suppressed`` with the value from column of the same name in the row with same BIB in HOL project"
            }},
    """
    For_OpenRefine.write(Template_String)
    for TKR_Field_Number in range(len(TKR_Fields)):
        Template_String = f"""
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "row-based"
                }},
                "baseColumnName": "SYS Number",
                "expression": "grel:cell.cross(\\"{HOL_Project_Name}\\",\\"BIB Number\\").cells[\\"TKR {TKR_Field_Number+1}\\"].value[0]",
                "onError": "set-to-blank",
                "newColumnName": "TKR {TKR_Field_Number+1}",
                "columnInsertIndex": {i+TKR_Field_Number+3},
                "description": "Create column ``TKR {TKR_Field_Number+1}`` with the value from the TKR column of the same number from row with same BIB in HOL project"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

**Using join multi-value cells on HOL and sublibrary columns removes suppressed status and TKRs from all but first HOL**